# Speech To Text with LLM

## Feature Extraction - Audio To Text

### Environment Preparation

In [ ]:
# Import necessary libraries and modules

# Operating System module - provides functions for interacting with the operating system
import os

# IPython display module to play audio in Jupyter notebooks
from IPython.display import Audio

# Hugging Face library for datasets - helps load and process datasets
from datasets import load_dataset

# Transformer library to load and use the Whisper model for audio-to-text tasks
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Librosa - a library for analyzing audio and music; provides audio processing tools
import librosa

# PyTorch - a deep learning framework used for managing model computations and GPU compatibility
import torch

# System-specific parameters and functions, e.g., to modify the Python path
import sys


# Set cache locations for Hugging Face and datasets

# Retrieve the Hugging Face cache directory from the environment variable and convert it to an absolute path.
# This path is used to store pre-trained models to avoid downloading them multiple times.
huggingface_cache_location = r"../../resources/huggingface_cache"

# # Retrieve the datasets cache directory from the environment variable and convert it to an absolute path.
# # This is the location where datasets are cached after initial download, helping save bandwidth and time.
# datasets_cache_location = os.path.abspath("../../../" + os.getenv("datasets_cache_location"))

# Set up the parent directory for custom modules

# Define the current directory path. This is useful if you have a custom module or scripts located in a current
# directory that you want to import into this script.
current_dir = os.path.abspath('./')

# Add the parent directory path to the Python path (sys.path), which allows importing modules from it.
sys.path.append(current_dir)

# Import any custom utilities located in the parent directory
# This step is optional and depends on whether you have additional scripts, such as helper functions, in the parent directory.
import local_utilities

data_helper = local_utilities.DataHelper()

# Set device to GPU if available, otherwise default to CPU

# Check if a CUDA-compatible GPU is available, and if so, set the device to GPU.
# This enables faster model processing by utilizing the GPU for heavy computations.
# If no GPU is available, the device will be set to CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Function Definitions

In [ ]:
import numpy as np
import pandas as pd
import psychai.feature.feature_extraction.feature_processor
import cv2
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas


class SpeechToTextExtractor:

    def __init__(self, processor, model):
        self.processor = processor
        self.model = model
        pass

    # def summarized_process_folder(self,input_file_fullname):

    #     # Extract features using multiple strategies and plot them
    #     strategies = ['mean', 'variance', 'max', 'end_to_begin']
        
    #     try:
    #         features, feature_names = self.process_audio_with_file_path(input_file_fullname)
    #     except Exception as e:
    #         print(f"Error: {e}")
    #         return None, None   

    #     # Summarize features using the specified strategies
    #     processor = psychai.feature.feature_extraction.feature_processor.FeatureProcessor()
    #     #processor = FeatureProcessor()
    #     summarized_features, summarized_feature_names = processor.summarize_features(features, strategies, feature_names)

    #     return summarized_features, summarized_feature_names  

    def process_audio_with_file_path(self, file_path):

        # 加载WAV文件
        # 使用librosa.load函数加载音频文件。设置sr=None以保持音频的原始采样率
        signal, sr = librosa.load(file_path, sr=None)  # sr=None 保留原始采样率

        # 对音频信号进行重采样
        # 将音频信号从原始采样率重采样到16000Hz，这是因为Whisper模型需要固定的采样率
        resampled_signal = librosa.resample(signal, orig_sr=sr, target_sr=16000)

        return self.process_audio_as_array(resampled_signal)

    def process_audio_as_array(self, sample):

        # 使用处理器处理音频样本，这里需要传入音频数组和采样率，并将结果转换为PyTorch张量
        input_features = self.processor(sample, sampling_rate=16000, return_tensors="pt").input_features 

        # 使用模型生成预测的token ids
        predicted_ids = self.model.generate(input_features)
        # 将token ids解码为文本
        transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=False)
        # 打印转录文本
        print(transcription)

        return np.array([transcription]), ['transcription']

    def process_audio_from_datasets(self, sample):

        # 使用处理器处理音频样本，这里需要传入音频数组和采样率，并将结果转换为PyTorch张量
        input_features = self.processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

        # 使用模型生成预测的token ids
        predicted_ids = self.model.generate(input_features)
        # 将token ids解码为文本
        transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=False)
        # 打印转录文本
        print(transcription)

        return transcription

        # # 将重采样后的音频信号处理为模型的输入特征
        # # 使用Whisper处理器处理重采样后的信号，并将采样率设置为16000Hz。return_tensors="pt"表明返回的是PyTorch张量
        # input_features = self.processor(signal, sampling_rate=16000, return_tensors="pt").input_features 

        # # 生成token id
        # # 使用模型的generate方法根据输入特征生成预测的token id
        # predicted_ids = self.model.generate(input_features)

        # # 将token id解码为文本
        # # 使用处理器的batch_decode方法将预测的token id解码成文本，skip_special_tokens=False表明不跳过特殊token
        # transcription = self.processor.batch_decode(predicted_ids, skip_special_tokens=False)

        # # 打印转录结果
        # return transcription

# # Define a function to clean up the text in each row
# def clean_text(text):
#     # Check if both markers exist in the text
#     start_marker = "<|notimestamps|>"
#     end_marker = "<|endoftext|>"
    
#     if start_marker in text and end_marker in text:
#         # Find the start and end positions of the markers
#         start_idx = text.find(start_marker) + len(start_marker)
#         end_idx = text.find(end_marker)
        
#         # Extract the text between the markers
#         return text[start_idx:end_idx].strip()  # .strip() to remove any leading/trailing spaces
#     else:
#         # If markers are not found, return the original text
#         return text
    
# # Group by user_id
# def calculate_mean_by_user_id(df):
#     # Step 1: Check consistency in non-numerical columns
#     df_consistent = df.groupby('user_id').apply(check_non_numerical_consistency).reset_index(drop=True)

#     # Step 2: Calculate mean for numerical columns
#     df_mean = df.groupby('user_id').mean(numeric_only=True).reset_index()

#     # Step 3: Add back consistent non-numerical columns
#     # Ensure 'user_id' is present in non_numerical_columns subset
#     non_numerical_columns = df_consistent[['user_id'] + df_consistent.select_dtypes(include=['object']).columns.tolist()].drop_duplicates(subset=['user_id'])
    
#     df_mean = pd.merge(df_mean, non_numerical_columns, on='user_id', how='left')

#     return df_mean


# # Group by user_id
# def calculate_mean_by_user_id(df):
#     # Step 1: Check consistency in non-numerical columns
#     df_consistent = df.groupby('user_id').apply(check_non_numerical_consistency).reset_index(drop=True)

#     # Step 2: Calculate mean for numerical columns
#     df_mean = df.groupby('user_id').mean(numeric_only=True).reset_index()

#     # Step 3: Add back consistent non-numerical columns
#     # Ensure 'user_id' is present in non_numerical_columns subset
#     non_numerical_columns = df_consistent[['user_id'] + df_consistent.select_dtypes(include=['object']).columns.tolist()].drop_duplicates(subset=['user_id'])
    
#     df_mean = pd.merge(df_mean, non_numerical_columns, on='user_id', how='left')

#     return df_mean

# # Function to check consistency in non-numerical columns
# def check_non_numerical_consistency(group):
#     for col in group.select_dtypes(include=['object']).columns:
#         if group[col].nunique() > 1:
#             raise ValueError(f"Inconsistent values found in column '{col}' for user_id '{group.name}'")
#     return group.iloc[0]  # Return the first row for consistent non-numerical columns


### Initialization

In [26]:
# 从预训练的模型中加载Whisper处理器和模型
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2", cache_dir=huggingface_cache_location)
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2", cache_dir=huggingface_cache_location)
model.config.forced_decoder_ids = None  # 确保模型不会强制使用解码器ID

# 加载一个示例数据集
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation", cache_dir=datasets_cache_location)



KeyboardInterrupt: 

In [ ]:
speech_to_text_extractor = SpeechToTextExtractor(processor, model)

### Audio To Text

#### English Example

这段代码首先处理了数据集中的第一个音频样本，将其转换为模型可以接受的格式，然后使用Whisper模型进行预测，并将预测结果（token ids）解码为可读的文本形式。最后，打印出转录的文本。注意，这里skip_special_tokens=False参数控制是否跳过特殊token，在解码时可以根据需要调整。

In [ ]:
# 获取第一个样本的音频信息
sample = ds[0]["audio"]
# 使用处理器处理音频样本，这里需要传入音频数组和采样率，并将结果转换为PyTorch张量
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features 

# 使用模型生成预测的token ids
predicted_ids = model.generate(input_features)
# 将token ids解码为文本
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
# 打印转录文本
print(transcription)

In [ ]:
# 获取第一个样本的音频信息
sample = ds[0]["audio"]
transcription = speech_to_text_extractor.process_audio_from_datasets(sample)

#### Chinese Example

In [ ]:
# Correct path to your audio file
audio_file_path = r"../resources/raw_data/005/Segment_Audio/PS-9_005_26_05_25_13_21_36.wav"
IPython.display.Audio(audio_file_path)

In [ ]:
# 加载WAV文件
# 使用librosa.load函数加载音频文件。设置sr=None以保持音频的原始采样率
signal, sr = librosa.load(audio_file_path, sr=None)  # sr=None 保留原始采样率

# 对音频信号进行重采样
# 将音频信号从原始采样率重采样到16000Hz，这是因为Whisper模型需要固定的采样率
resampled_signal = librosa.resample(signal, orig_sr=sr, target_sr=16000)

# 将重采样后的音频信号处理为模型的输入特征
# 使用Whisper处理器处理重采样后的信号，并将采样率设置为16000Hz。return_tensors="pt"表明返回的是PyTorch张量
input_features = processor(resampled_signal, sampling_rate=16000, return_tensors="pt").input_features 

# 生成token id
# 使用模型的generate方法根据输入特征生成预测的token id
predicted_ids = model.generate(input_features)

# 将token id解码为文本
# 使用处理器的batch_decode方法将预测的token id解码成文本，skip_special_tokens=False表明不跳过特殊token
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

# 打印转录结果
print(transcription)

In [ ]:
transcription = speech_to_text_extractor.process_audio_with_file_path(audio_file_path)

#### File Folder and Class Example

In [ ]:
#base_directory = r"D:\Programming3\psychai\example\paper5_mmer_moral\resources\raw_data"
base_directory = r"G:\Experiments\Moral Elevation\Disk1_2_combined\FS-9 Day 1"

# Create a FileFilter instance with multiple attribute filters
filters = [(2, '25'),(2, '26'),(2, '27'),(2, '28')]
file_filter = utilities.FileFilter(base_directory, attribute_filters=filters, modality_value='Segment_Audio')

# Get matching files for user IDs from 1 to 13
df = file_filter.get_matching_files(range(1, 120))

# Load the rct.csv file
rct_df = pd.read_csv('../resources/data/rct/rct.csv')

# Merge the two dataframes based on the 'user_id' column
df_updated = pd.merge(df, rct_df[['user_id', 'Group']], on='user_id', how='left')

# Find the row with the largest file_size for each user_id
# df_largest = df_updated.loc[df_updated.groupby('user_id')['file_size'].idxmax()]
df_largest = df_updated.loc[df_updated.groupby(['user_id', 'attribute_value'])['file_size'].idxmax()]

# Reset the index if needed
df_largest.reset_index(drop=True, inplace=True)

# Display the result
print(df_largest)

Segment_Audio folder not found for user 010
Segment_Audio folder not found for user 025
Segment_Audio folder not found for user 026
Segment_Audio folder not found for user 027
Segment_Audio folder not found for user 042
Segment_Audio folder not found for user 071
Segment_Audio folder not found for user 074
Segment_Audio folder not found for user 076
Segment_Audio folder not found for user 077
Segment_Audio folder not found for user 088
Segment_Audio folder not found for user 091
Segment_Audio folder not found for user 112
Segment_Audio folder not found for user 116
Segment_Audio folder not found for user 118
Segment_Audio folder not found for user 119
Total matching files found: 568
     user_id       modality  attribute_index attribute_value  \
0          1  Segment_Audio                2              25   
1          1  Segment_Audio                2              26   
2          1  Segment_Audio                2              27   
3          1  Segment_Audio                2        

In [ ]:
import os
import pandas as pd
import pickle
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# Path to cache folder
cache_folder = r"D:\Programming3\psychai\example\paper5_mmer_moral\results\speech_text\cache"
os.makedirs(cache_folder, exist_ok=True)  # Ensure cache folder exists

# Function to process each file with caching
def process_file(row):
    audio_file_path = row['files']
    cache_file_path = os.path.join(cache_folder, f"{os.path.basename(audio_file_path)}.pkl")
    
    # Check if the cache file exists
    if os.path.exists(cache_file_path):
        # Load the cached result
        with open(cache_file_path, 'rb') as cache_file:
            text = pickle.load(cache_file)
    else:
        # Run the speech-to-text processing and save to cache
        text = speech_to_text_extractor.process_audio_with_file_path(audio_file_path)
        text = clean_text(text)
        
        # Save the result to a cache file
        with open(cache_file_path, 'wb') as cache_file:
            pickle.dump(text, cache_file)

    # Return the text result for adding to DataFrame
    return text

# Apply parallel processing with progress tracking
def process_files_in_parallel(df):
    # Add a "text" column to the DataFrame to store results
    df['text'] = None
    
    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(process_file, row): idx for idx, row in df.iterrows()}
        
        # Use tqdm for a progress bar
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing files"):
            idx = futures[future]
            try:
                print(f"[file {idx}] starts.")
                # Update the 'text' column with the result
                df.at[idx, 'text'] = future.result()
                print(f"[file {idx}] completed.")
            except Exception as e:
                print(f"Error processing file at index {idx}: {e}")

    return df

# Run the processing function
df_largest = process_files_in_parallel(df_largest)
df_largest.to_csv(r"D:\Programming3\psychai\example\paper5_mmer_moral\results\speech_text\csv\result_20241107.csv")

Processing files:   0%|          | 0/404 [00:00<?, ?it/s]

[file 9] starts.
[file 9] completed.
[file 2] starts.
[file 2] completed.
[file 7] starts.
[file 7] completed.
[file 22] starts.
[file 22] completed.
[file 4] starts.
[file 4] completed.
[file 13] starts.
[file 13] completed.
[file 5] starts.
[file 5] completed.
[file 0] starts.
[file 0] completed.
[file 3] starts.
[file 3] completed.
[file 6] starts.
[file 6] completed.
[file 20] starts.
[file 20] completed.
[file 10] starts.
[file 10] completed.
[file 26] starts.
[file 26] completed.
[file 16] starts.
[file 16] completed.
[file 15] starts.
[file 15] completed.
[file 12] starts.
[file 12] completed.
[file 8] starts.
[file 8] completed.
[file 24] starts.
[file 24] completed.
[file 1] starts.
[file 1] completed.
[file 17] starts.
[file 17] completed.
[file 11] starts.
[file 11] completed.
[file 19] starts.
[file 19] completed.
[file 18] starts.
[file 18] completed.
[file 14] starts.
[file 14] completed.
[file 25] starts.
[file 25] completed.
[file 21] starts.
[file 21] completed.
[file 

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Processing files:  31%|███       | 125/404 [07:23<16:29,  3.55s/it]

['<|startoftranscript|><|zh|><|transcribe|><|notimestamps|>沒有因為我已經看過這個了對<|endoftext|>']
[file 133] starts.
[file 133] completed.


Processing files:  31%|███       | 126/404 [07:55<17:55,  3.87s/it]

...

['<|startoftranscript|><|zh|><|transcribe|><|notimestamps|>有让我有一种就是我现在很想给我爸爸或者妈妈打一个电话然后就是我应该要珍惜跟自己家人在一起的时光然后跟自己周围人在一起的时光因为影片所传达的一个意义就是说你应该要跟你爱的人<|endoftext|>']
[file 395] starts.
[file 395] completed.


Processing files: 100%|█████████▉| 403/404 [2:33:27<00:03,  3.93s/it]

['<|startoftranscript|><|zh|><|transcribe|><|notimestamps|>我觉得最强烈的印象应该是影片中的最后一幕她哥哥被打倒但是她看见哭泣的妹妹还是站起来然后变结然后看见哭泣的妹妹还有以前的房坝就逗她笑<|endoftext|>']
[file 396] starts.
[file 396] completed.


Processing files: 100%|██████████| 404/404 [2:33:29<00:00, 22.80s/it]

['<|startoftranscript|><|zh|><|transcribe|><|notimestamps|>我觉得这世上的人有好有坏吧就像是人之初性本善大家可能都是好的然后就是出生下来都是一张白纸但是经过后天的环境的影响然后可能对性格的塑造方面所以就是很不相同的然后<|endoftext|>']
[file 398] starts.
[file 398] completed.


## Footnote
- Copyright：Ivan Liu 
- Last Update: 2024
- Env：psychai241104
- References: 
    - None